In [1]:
import pandas as pd
import numpy as np
import math

In [ ]:
df = pd.read_csv("../data/1M-LdSt.csv")

In [ ]:
df = df.replace('NAN', np.nan)
df['pc'] = df['pc'].astype('int64')
df['effective_address'] = df['effective_address'].astype('int64')
df['val0'] = df['val0'].astype('int64')

df1 = pd.DataFrame(index=df[df['type'] == 'l'].index)
df1['pc-l'] = df['pc'] % math.pow(2, 31) # lower 32 bits
#df1['pc-u'] = df['pc'] - df1['pc-l']     # first 32 bits (leave out)
df1['ea-l'] = df['effective_address'] % math.pow(2, 31)
df1['ea-u'] = df['effective_address'] - df1['ea-l']
df1['v0-l'] = df['val0'] % math.pow(2, 31)
df1['v0-u'] = df['val0'] - df1['v0-l']

pcl_range = df1['pc-l'].max() #- df1['pc-l'].min()
#pcu_range = df1['pc-u'].max() - df1['pc-u'].min()
eal_range = df1['ea-l'].max() #- df1['ea-l'].min()
eau_range = df1['ea-u'].max() #- df1['ea-u'].min()
v0l_range = df1['v0-l'].max() #- df1['v0-l'].min()
v0u_range = df1['v0-u'].max() #- df1['v0-u'].min()

df1['pc-l'] = df1['pc-l']/pcl_range
#df1['pc-u'] = df1['pc-u']/pcu_range
df1['ea-l'] = df1['ea-l']/eal_range
df1['ea-u'] = df1['ea-u']/eau_range
df1['v0-l'] = df1['v0-l']/v0l_range
df1['v0-u'] = df1['v0-u']/v0u_range

In [ ]:
df1

In [ ]:
# Add columns for the previous k loads
k=4
for i in range(1, k+1):
    lname = 'v0l-' + str(i)
    df1[lname] = df1['v0-l'].shift(i)
    uname = 'v0u-' + str(i)
    df1[uname] = df1['v0-u'].shift(i)

In [ ]:
def get_previous_store_lower(i, dfs):
    """ Helper method used in prep_dataFrame.
        We assume df has a column called 'v0-l' and that if the 
        index is j then row j contains the data for the j^th
        instruction. We assume that df contains only the stores.
        
        We grab the lower 32 (least significant) bits of the
        first store which occurs prior to the i^th instruction
        and return this value.
    """
    idx = dfs[dfs.index < i].index.max()
    if (pd.isnull(idx)):
        return np.nan
    return dfs.loc[idx, 'v0-l']

def get_previous_store_upper(i, dfs):
    idx = dfs[dfs.index < i].index.max()
    if (pd.isnull(idx)):
        return np.nan
    return dfs.loc[idx, 'v0-u']

In [ ]:
# Add columns for the first 32 and last 32 bits of the previous store value
# dfs = pd.DataFrame(index=df[df['type'] == 's'].index)
# dfs['v0-l'] = df['val0'] % math.pow(2, 31)
# dfs['v0-u'] = df['val0'] - dfs['v0-l']
# sv0l_range = dfs['v0-l'].max()
# sv0u_range = dfs['v0-u'].max()
# df1['s-1l'] = df1.index.to_series().apply(lambda x : get_previous_store_lower(x, dfs))
# df1['s-1u'] = df1.index.to_series().apply(lambda x : get_previous_store_upper(x, dfs))

In [ ]:
# df1['s-1l'] = df1['s-1l']/sv0l_range
# df1['s-1u'] = df1['s-1u']/sv0u_range 

In [ ]:
df1

In [ ]:
df1 = df1[["pc-l", "ea-l", "ea-u", "v0l-1", "v0u-1", "v0l-2", "v0u-2", "v0l-3", "v0u-3", "v0l-4", "v0u-4", "s-1l", "s-1u", "v0-l", "v0-u"]]

In [ ]:
df1 = df1.dropna()

In [ ]:
df1

In [1]:
import sys  
sys.path.insert(0, '../MLforVP2')

In [2]:
from dnns import *

In [3]:
dff = prepDataFrame("1M-LdSt.csv", read_stores=True)

In [4]:
#dff[ dff[dff['s-1u'] < 1.0] > 0.0 ]
dff

,pc-l,ea-l,ea-u,v0l-1,v0u-1,v0l-2,v0u-2,v0l-3,v0u-3,v0l-4,v0u-4,s-1l,s-1u,v0-l,v0-u
11,0.808478,0.032534,0.0,3.253402e-02,0.000000,4.703179e-08,0.000000,9.999916e-01,0.000031,2.158662e-02,0.000000,0.999992,1.0,1.586942e-03,0.000000
12,0.808481,0.004165,0.0,1.586942e-03,0.000000,3.253402e-02,0.000000,4.703179e-08,0.000000,9.999916e-01,0.000031,0.999992,1.0,3.150437e-03,0.000000
13,0.808483,1.000000,1.0,3.150437e-03,0.000000,1.586942e-03,0.000000,3.253402e-02,0.000000,4.703179e-08,0.000000,0.999992,1.0,3.253406e-02,0.000000
14,0.808484,1.000000,1.0,3.253406e-02,0.000000,3.150437e-03,0.000000,1.586942e-03,0.000000,3.253402e-02,0.000000,0.999992,1.0,4.332530e-03,0.000000
15,0.808484,1.000000,1.0,4.332530e-03,0.000000,3.253406e-02,0.000000,3.150437e-03,0.000000,1.586942e-03,0.000000,0.999992,1.0,2.158662e-02,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
262099,0.999946,1.000000,1.0,2.158709e-02,0.000000,0.000000e+00,0.000000,9.670496e-03,0.000000,9.670377e-03,0.000000,0.009670,0.0,8.927360e-03,0.000000
262100,0.999946,1.000000,1.0,8.927360e-03,0.000000,2.158709e-02,0.000000,0.000000e+00,0.000000,9.670496e-03,0.000000,0.009670,0.0,1.443550e-08,0.000000
262101,0.999947,1.000000,1.0,1.443550e-08,0.000000,8.927360e-03,0.000000,2.158709e-02,0.000000,0.000000e+00,0.000000,0.009670,0.0,9.999916e-01,0.000031
262103,0.572340,0.004870,0.0,9.999916e-01,0.000031,1.443550e-08,0.000000,8.927360e-03,0.000000,2.158709e-02,0.000000,0.002230,0.0,2.710940e-05,0.000000


In [5]:
X , Y = input_response_split(dff)

In [6]:
X

array([[8.08478144e-01, 3.25342892e-02, 0.00000000e+00, ...,
        0.00000000e+00, 9.99991536e-01, 1.00000000e+00],
       [8.08481490e-01, 4.16480494e-03, 0.00000000e+00, ...,
        3.06370212e-05, 9.99991536e-01, 1.00000000e+00],
       [8.08483402e-01, 9.99999873e-01, 1.00000000e+00, ...,
        0.00000000e+00, 9.99991536e-01, 1.00000000e+00],
       ...,
       [9.99946942e-01, 9.99999888e-01, 1.00000000e+00, ...,
        0.00000000e+00, 9.67049226e-03, 0.00000000e+00],
       [5.72340464e-01, 4.87041433e-03, 0.00000000e+00, ...,
        0.00000000e+00, 2.23039836e-03, 0.00000000e+00],
       [5.72343332e-01, 4.87136428e-03, 0.00000000e+00, ...,
        0.00000000e+00, 2.71094032e-05, 0.00000000e+00]])

In [7]:
Y

array([[1.58694247e-03, 0.00000000e+00],
       [3.15043703e-03, 0.00000000e+00],
       [3.25340629e-02, 0.00000000e+00],
       ...,
       [9.99991596e-01, 3.06370212e-05],
       [2.71094032e-05, 0.00000000e+00],
       [5.36842346e-02, 0.00000000e+00]])

In [ ]:
def test_train_split(df, num_training_examples):
    """ Splits the dataframe into a test set and training set. Uses
        input_response_split to produce X, y then splits these into
        training and testing sets.
        usage: 
        X_tr, X_test, y_tr, y_test = test_train_split(df, num_training_examples)
    """
    assert(num_training_examples < len(df))
    X, y = input_response_split(df)
    X_tr = X[:num_training_examples,]
    X_te  = X[num_training_examples:,]
    y_tr = y[:num_training_examples]
    y_te  = y[num_training_examples:]
    return X_tr, X_te, y_tr, y_te 

In [1]:
import sys  
sys.path.insert(0, '../src')
from dnns import *

In [2]:
df1 = prepDataFrame("../data/1M-LdSt.csv", read_stores=False)
#df1 = prepDataFrame("../data/1M-LdSt.csv")

In [3]:
df1

,pc-l,ea-l,ea-u,v0l-1,v0u-1,v0l-2,v0u-2,v0l-3,v0u-3,v0l-4,v0u-4,s-1l,s-1u,s-2l,s-2u,v0-l,v0-u
11,0.808478,0.032534,0.0,3.253402e-02,0.000000,4.703179e-08,0.000000,9.999916e-01,0.000031,2.158662e-02,0.000000,0.999992,1.0,0.999992,1.0,1.586942e-03,0.000000
12,0.808481,0.004165,0.0,1.586942e-03,0.000000,3.253402e-02,0.000000,4.703179e-08,0.000000,9.999916e-01,0.000031,0.999992,1.0,0.999992,1.0,3.150437e-03,0.000000
13,0.808483,1.000000,1.0,3.150437e-03,0.000000,1.586942e-03,0.000000,3.253402e-02,0.000000,4.703179e-08,0.000000,0.999992,1.0,0.999992,1.0,3.253406e-02,0.000000
14,0.808484,1.000000,1.0,3.253406e-02,0.000000,3.150437e-03,0.000000,1.586942e-03,0.000000,3.253402e-02,0.000000,0.999992,1.0,0.999992,1.0,4.332530e-03,0.000000
15,0.808484,1.000000,1.0,4.332530e-03,0.000000,3.253406e-02,0.000000,3.150437e-03,0.000000,1.586942e-03,0.000000,0.999992,1.0,0.999992,1.0,2.158662e-02,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
262099,0.999946,1.000000,1.0,2.158709e-02,0.000000,0.000000e+00,0.000000,9.670496e-03,0.000000,9.670377e-03,0.000000,0.009670,0.0,0.009670,0.0,8.927360e-03,0.000000
262100,0.999946,1.000000,1.0,8.927360e-03,0.000000,2.158709e-02,0.000000,0.000000e+00,0.000000,9.670496e-03,0.000000,0.009670,0.0,0.009670,0.0,1.443550e-08,0.000000
262101,0.999947,1.000000,1.0,1.443550e-08,0.000000,8.927360e-03,0.000000,2.158709e-02,0.000000,0.000000e+00,0.000000,0.009670,0.0,0.009670,0.0,9.999916e-01,0.000031
262103,0.572340,0.004870,0.0,9.999916e-01,0.000031,1.443550e-08,0.000000,8.927360e-03,0.000000,2.158709e-02,0.000000,0.002230,0.0,0.009670,0.0,2.710940e-05,0.000000


In [5]:
df1

,pc-l,ea-l,ea-u,v0l-1,v0u-1,v0l-2,v0u-2,v0l-3,v0u-3,v0l-4,v0u-4,s-1l,s-1u,s-2l,s-2u,v0-l,v0-u
11,0.808478,0.032534,0.0,3.253402e-02,0.000000,4.703179e-08,0.000000,9.999916e-01,0.000031,2.158662e-02,0.000000,0.999992,1.0,0.999992,1.0,1.586942e-03,0.000000
12,0.808481,0.004165,0.0,1.586942e-03,0.000000,3.253402e-02,0.000000,4.703179e-08,0.000000,9.999916e-01,0.000031,0.999992,1.0,0.999992,1.0,3.150437e-03,0.000000
13,0.808483,1.000000,1.0,3.150437e-03,0.000000,1.586942e-03,0.000000,3.253402e-02,0.000000,4.703179e-08,0.000000,0.999992,1.0,0.999992,1.0,3.253406e-02,0.000000
14,0.808484,1.000000,1.0,3.253406e-02,0.000000,3.150437e-03,0.000000,1.586942e-03,0.000000,3.253402e-02,0.000000,0.999992,1.0,0.999992,1.0,4.332530e-03,0.000000
15,0.808484,1.000000,1.0,4.332530e-03,0.000000,3.253406e-02,0.000000,3.150437e-03,0.000000,1.586942e-03,0.000000,0.999992,1.0,0.999992,1.0,2.158662e-02,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
262099,0.999946,1.000000,1.0,2.158709e-02,0.000000,0.000000e+00,0.000000,9.670496e-03,0.000000,9.670377e-03,0.000000,0.009670,0.0,0.009670,0.0,8.927360e-03,0.000000
262100,0.999946,1.000000,1.0,8.927360e-03,0.000000,2.158709e-02,0.000000,0.000000e+00,0.000000,9.670496e-03,0.000000,0.009670,0.0,0.009670,0.0,1.443550e-08,0.000000
262101,0.999947,1.000000,1.0,1.443550e-08,0.000000,8.927360e-03,0.000000,2.158709e-02,0.000000,0.000000e+00,0.000000,0.009670,0.0,0.009670,0.0,9.999916e-01,0.000031
262103,0.572340,0.004870,0.0,9.999916e-01,0.000031,1.443550e-08,0.000000,8.927360e-03,0.000000,2.158709e-02,0.000000,0.002230,0.0,0.009670,0.0,2.710940e-05,0.000000
